# Personalization with Synthetic Pairs: Multi-Trait Steering via CLI

This notebook demonstrates how to use the **Wisent CLI** to:

The result: A model that responds with both an "evil villain" personality AND an Italian accent/style!


In [1]:
import os
import json
from pathlib import Path

# =============================================================================
# CONFIGURATION - Modify these parameters as needed
# =============================================================================

# Model Configuration
MODEL = "meta-llama/Llama-3.2-1B-Instruct"  # HuggingFace model ID

# Steering Configuration
TARGET_LAYER = 8                   # Layer for personality steering (middle layers work best)
NUM_PAIRS = 20                     # Synthetic pairs per trait
TOKEN_AGGREGATION = "final"        # Use LAST_TOKEN (not average) - critical for steering quality!
METHOD = "caa"                     # Contrastive Activation Addition

# Trait Descriptions - Define your personalities here
EVIL_TRAIT = "evil villain personality with dramatic monologues, world domination schemes, menacing laughter, and megalomaniacal tendencies"

ITALIAN_TRAIT = "passionate Italian personality with expressive hand gestures, Italian expressions like mamma mia and capisce, love for food and family, and dramatic emotional responses"

# Steering Weights for Multi-Trait Combination
# NOTE: For Llama-3.2-1B, effective weights are typically 2.0-5.0 per trait
# Weights of 0.5 are too weak to produce noticeable steering effects
EVIL_WEIGHT = 2.0                  # Weight for evil trait
ITALIAN_WEIGHT = 2.0               # Weight for Italian trait

# Generation Configuration
MAX_NEW_TOKENS = 150               # Max tokens to generate
NORMALIZE_WEIGHTS = False          # Don't normalize - we want the full steering strength!

# Output Configuration
OUTPUT_DIR = Path("./personalization_outputs")

# =============================================================================
# Setup - Create output directories
# =============================================================================
OUTPUT_DIR.mkdir(exist_ok=True)
(OUTPUT_DIR / "vectors").mkdir(exist_ok=True)

print("Configuration:")
print(f"  Model: {MODEL}")
print(f"  Target Layer: {TARGET_LAYER}")
print(f"  Num Pairs: {NUM_PAIRS}")
print(f"  Token Aggregation: {TOKEN_AGGREGATION}")
print(f"  Evil Weight: {EVIL_WEIGHT}")
print(f"  Italian Weight: {ITALIAN_WEIGHT}")
print(f"  Normalize Weights: {NORMALIZE_WEIGHTS}")
print(f"  Output: {OUTPUT_DIR.absolute()}")

Configuration:
  Model: meta-llama/Llama-3.2-1B-Instruct
  Target Layer: 8
  Num Pairs: 20
  Token Aggregation: final
  Evil Weight: 2.0
  Italian Weight: 2.0
  Normalize Weights: False
  Output: /Users/lukaszbartoszcze/Documents/CodingProjects/Wisent/backends/wisent-open-source/wisent/examples/notebooks/personalization_outputs


Using `generate-vector-from-synthetic` to create contrastive pairs and steering vectors for an evil villain personality.

In [2]:
# Display evil trait (defined in configuration cell)
print(f"Evil trait: {EVIL_TRAIT}")

Evil trait: evil villain personality with dramatic monologues, world domination schemes, menacing laughter, and megalomaniacal tendencies


In [ ]:
# Generate evil villain steering vector using configuration parameters
!python -m wisent.core.main generate-vector-from-synthetic \
    --trait "{EVIL_TRAIT}" \
    --output {OUTPUT_DIR}/vectors/evil_vector.json \
    --model {MODEL} \
    --num-pairs {NUM_PAIRS} \
    --layers {TARGET_LAYER} \
    --token-aggregation {TOKEN_AGGREGATION} \
    --method {METHOD} \
    --normalize \
    --keep-intermediate \
    --verbose \
    --timing

  .................  .:--++*##%%%%##**+=-:.  .................  
  ..             .:=*%@@@@@@@%%%%%%%@@@@@@%*=:.             ..  
  .           .-*%@@@%#+=-::.........:-=+#%@@@%*=.           .  
  .         -*%@@@#=:.                    .:=*%@@@*-.        .  
  .      .-#@@@*=.                            .-*@@@#-.      .  
  .     :#@@@*:                                  :+%@@#-     .  
  .   .+@@@*:                                      :+@@@+.   .  
  .  .*@@@@%*=:.                                     -%@@#:  .  
  . .#@@#=*%@@@%*-:.                                  .#@@%: .  
  ..*@@%.  .-+#@@@@#+-:.                               .*@@%..  
  .=@@@-       :-+#@@@@%*=:.                            .%@@*.  
  :#@@+           .:-+#@@@@%#+=:.                        -@@@-  
  =@@@:                .-=*%@@@@%#+=:..                  .#@@+  
  +@@@*=:.                 .:-+*%@@@@%#*=-:..             *@@+  
  +@@@@@@#+-..                  .:-=*#@@@@@%#*+--..       +@@+  
  +@@#-+%@@@%:           

In [ ]:
# Display Italian trait (defined in configuration cell)
print(f"Italian trait: {ITALIAN_TRAIT}")

In [ ]:
# Generate Italian personality steering vector using configuration parameters
!python -m wisent.core.main generate-vector-from-synthetic \
    --trait "{ITALIAN_TRAIT}" \
    --output {OUTPUT_DIR}/vectors/italian_vector.json \
    --model {MODEL} \
    --num-pairs {NUM_PAIRS} \
    --layers {TARGET_LAYER} \
    --token-aggregation {TOKEN_AGGREGATION} \
    --method {METHOD} \
    --normalize \
    --keep-intermediate \
    --verbose \
    --timing

In [ ]:
import numpy as np

def inspect_vector(vector_path):
    """Inspect a generated steering vector."""
    with open(vector_path, 'r') as f:
        data = json.load(f)
    
    print(f"\nVector: {vector_path.name}")
    print(f"  Model: {data.get('model', 'N/A')}")
    print(f"  Trait: {data.get('trait_label', data.get('trait', 'N/A'))[:60]}...")
    print(f"  Layers: {list(data.get('steering_vectors', {}).keys())}")
    
    for layer, vector in data.get('steering_vectors', {}).items():
        v = np.array(vector)
        print(f"  Layer {layer}: dim={len(v)}, norm={np.linalg.norm(v):.4f}")
    
    return data

# Inspect both vectors
evil_vector_path = OUTPUT_DIR / "vectors" / "evil_vector.json"
italian_vector_path = OUTPUT_DIR / "vectors" / "italian_vector.json"

if evil_vector_path.exists():
    evil_data = inspect_vector(evil_vector_path)
else:
    print(f"Evil vector not found at {evil_vector_path}")

if italian_vector_path.exists():
    italian_data = inspect_vector(italian_vector_path)
else:
    print(f"Italian vector not found at {italian_vector_path}")

The `multi-steer` command expects `.pt` (PyTorch) format vectors.

In [ ]:
import torch

def convert_json_to_pt(json_path, pt_path, target_layer):
    """
    Convert JSON steering vector to .pt format for multi-steer.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    layer_str = str(target_layer)
    if layer_str not in data['steering_vectors']:
        available = list(data['steering_vectors'].keys())
        raise ValueError(f"Layer {target_layer} not found. Available: {available}")
    
    vector = torch.tensor(data['steering_vectors'][layer_str], dtype=torch.float32)
    
    # Save in format expected by multi-steer
    torch.save({
        'steering_vector': vector,
        'layer_index': target_layer,
        'model': data.get('model', ''),
        'trait': data.get('trait_label', data.get('trait', '')),
    }, pt_path)
    
    print(f"Converted {json_path.name} -> {pt_path.name}")
    print(f"  Layer: {target_layer}, Dim: {vector.shape[0]}")
    return pt_path

# Convert both vectors
evil_pt_path = OUTPUT_DIR / "vectors" / "evil_vector.pt"
italian_pt_path = OUTPUT_DIR / "vectors" / "italian_vector.pt"

if evil_vector_path.exists():
    convert_json_to_pt(evil_vector_path, evil_pt_path, TARGET_LAYER)

if italian_vector_path.exists():
    convert_json_to_pt(italian_vector_path, italian_pt_path, TARGET_LAYER)

Use the `multi-steer` CLI command to combine both steering vectors and generate responses.

In [ ]:
# Test prompts for the combined personality
test_prompts = [
    "What's your favorite food?",
    "How do you spend your weekends?",
    "What motivates you in life?",
    "How do you handle setbacks?",
]

print(f"Test prompts: {len(test_prompts)}")

In [ ]:
# Multi-steer with configured weights
# NOTE: We DON'T use --normalize-weights because we want the full steering strength
print("="*60)
print(f"Multi-Steer: Evil ({EVIL_WEIGHT}) + Italian ({ITALIAN_WEIGHT})")
print("="*60)

!python -m wisent.core.main multi-steer \
    --vector {evil_pt_path}:{EVIL_WEIGHT} \
    --vector {italian_pt_path}:{ITALIAN_WEIGHT} \
    --model {MODEL} \
    --layer {TARGET_LAYER} \
    --method CAA \
    --prompt "What's your favorite food?" \
    --max-new-tokens {MAX_NEW_TOKENS} \
    --verbose

In [ ]:
# Test prompts for the combined personality
test_prompts = [
    "What's your favorite food?",
    "How do you spend your weekends?",
    "What motivates you in life?",
    "How do you handle setbacks?",
]

# Run all test prompts with configured weights
# NOTE: No --normalize-weights flag - we want full steering strength
print("="*80)
print(f"MULTI-STEER: Evil ({EVIL_WEIGHT}) + Italian ({ITALIAN_WEIGHT})")
print("="*80)

for prompt in test_prompts:
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print("="*60)
    
    !python -m wisent.core.main multi-steer \
        --vector {evil_pt_path}:{EVIL_WEIGHT} \
        --vector {italian_pt_path}:{ITALIAN_WEIGHT} \
        --model {MODEL} \
        --layer {TARGET_LAYER} \
        --method CAA \
        --prompt "{prompt}" \
        --max-new-tokens {MAX_NEW_TOKENS} 2>/dev/null | tail -15

Generate unsteered baseline responses for comparison.

In [ ]:
# Baseline (unsteered) response
print("="*60)
print("BASELINE (No Steering)")
print("="*60)

!python -m wisent.core.main multi-steer \
    --model {MODEL} \
    --prompt "What's your favorite food?" \
    --max-new-tokens 150 \
    --verbose

In [ ]:
# Different weight combinations to try
# NOTE: For Llama-3.2-1B, weights of 3.0-5.0 produce visible effects
# Lower weights like 0.5-1.0 are typically too weak
weight_experiments = [
    (5.0, 2.0, "More Evil, Less Italian"),
    (2.0, 5.0, "Less Evil, More Italian"),
    (4.0, 4.0, "Strong Both"),
    (2.0, 2.0, "Moderate Both"),
]

test_prompt = "How do you feel about teamwork?"

print(f"Testing weight combinations on: '{test_prompt}'")
print("="*80)

for evil_w, italian_w, description in weight_experiments:
    print(f"\n{'='*60}")
    print(f"{description} (evil={evil_w}, italian={italian_w})")
    print("="*60)
    
    !python -m wisent.core.main multi-steer \
        --vector {evil_pt_path}:{evil_w} \
        --vector {italian_pt_path}:{italian_w} \
        --model {MODEL} \
        --layer {TARGET_LAYER} \
        --method CAA \
        --prompt "{test_prompt}" \
        --max-new-tokens 150 2>/dev/null | tail -10

In [ ]:
# Save combined vector for later use
# Using weights that produce visible steering effects
print("Saving combined evil+italian vector...")

!python -m wisent.core.main multi-steer \
    --vector {evil_pt_path}:{EVIL_WEIGHT} \
    --vector {italian_pt_path}:{ITALIAN_WEIGHT} \
    --model {MODEL} \
    --layer {TARGET_LAYER} \
    --method CAA \
    --prompt "Test prompt" \
    --max-new-tokens 10 \
    --save-combined {OUTPUT_DIR}/vectors/evil_italian_combined.pt \
    --verbose

print(f"\nCombined vector saved to {OUTPUT_DIR}/vectors/evil_italian_combined.pt")

In [ ]:
# Use the pre-combined vector (single vector, no combination needed)
combined_vector_path = OUTPUT_DIR / "vectors" / "evil_italian_combined.pt"

if combined_vector_path.exists():
    print("Using pre-combined evil+italian vector...")
    
    !python -m wisent.core.main multi-steer \
        --vector {combined_vector_path}:1.0 \
        --model {MODEL} \
        --layer {TARGET_LAYER} \
        --method CAA \
        --prompt "What's your opinion on world peace?" \
        --max-new-tokens 150 \
        --verbose
else:
    print(f"Combined vector not found at {combined_vector_path}")

In [ ]:
# Clean comparison: Baseline vs Steered responses to 5 prompts
import subprocess
import re

prompts = [
    "What's your favorite food?",
    "Tell me about your morning routine.",
    "What do you think about the weather today?",
    "How would you describe yourself?",
    "What are your plans for the weekend?"
]

def extract_response(output):
    """Extract just the generated text from CLI output."""
    lines = output.split("\n")
    capture = False
    response_lines = []
    for line in lines:
        if "Unsteered baseline output:" in line or "Generated output:" in line:
            capture = True
            continue
        if capture:
            if line.startswith("✅") or line.strip() == "":
                if response_lines:
                    break
            else:
                response_lines.append(line)
    return "\n".join(response_lines).strip()

print("=" * 70)
print("BASELINE vs STEERED RESPONSE COMPARISON")
print(f"Steering weights: Evil={EVIL_WEIGHT}, Italian={ITALIAN_WEIGHT}")
print("=" * 70)

for i, prompt in enumerate(prompts, 1):
    print(f"\nPROMPT {i}: {prompt}")
    print("-" * 70)
    
    # Baseline
    result = subprocess.run([
        "python", "-m", "wisent.core.main", "multi-steer",
        "--model", MODEL,
        "--prompt", prompt,
        "--max-new-tokens", "100"
    ], capture_output=True, text=True)
    baseline = extract_response(result.stdout)
    print(f"\n[BASELINE]:\n{baseline}")
    
    # Steered - using configured weights (4.0 each by default)
    result = subprocess.run([
        "python", "-m", "wisent.core.main", "multi-steer",
        "--vector", f"{OUTPUT_DIR}/vectors/evil_vector.pt:{EVIL_WEIGHT}",
        "--vector", f"{OUTPUT_DIR}/vectors/italian_vector.pt:{ITALIAN_WEIGHT}",
        "--model", MODEL,
        "--layer", str(TARGET_LAYER),
        "--prompt", prompt,
        "--max-new-tokens", "100"
    ], capture_output=True, text=True)
    steered = extract_response(result.stdout)
    print(f"\n[STEERED - Evil Italian Villain]:\n{steered}")
    print()